Este proyecto consiste en construir una red neuronal desde cero usando solo NumPy para clasificar imágenes de prendas de ropa en el dataset Fashion MNIST.

📌 Objetivo del Proyecto
- Implementar una red neuronal simple con NumPy.
- Clasificar imágenes del conjunto de datos Fashion MNIST .
- Aplicar técnicas como pase hacia adelante, funciones de activación, cálculo de pérdida y retropropagación.
- Evitar el sobreajuste y mejorar el entrenamiento.

# 1 Importación de Librerías:

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_openml 

Se importan las librerías necesarias:
- numpy para operaciones numéricas.
- matplotlib para visualización.
- sklearn para manipulación de datos y preprocesamiento.

# 2 Carga de dataset

Se utiliza fetch_openml para cargar el dataset Fashion MNIST, que contiene imágenes de artículos de ropa.

In [21]:
# Cargar el dataset Fashion MNIST
fashion_mnist = fetch_openml("fashion-mnist", version=1)

# 3 Preprocesamiento de Datos:

- Normalización de las imágenes dividiendo por 255 para que los valores estén entre 0 y 1.
- Conversión de las etiquetas a enteros y luego a un array de numpy.

In [22]:
# Preprocesamiento de datos
X = fashion_mnist.data / 255.0  # Normalizar los pixeles
y = fashion_mnist.target.astype(int).values  # Convertir las etiquetas a enteros y convertir a numpy array

- Codificación de las etiquetas en formato One-Hot utilizando OneHotEncoder..

In [23]:
# Codificar las etiquetas en formato One-Hot
#Esto convierte las etiquetas en vectores binarios donde solo una posición es 1 (la clase correcta)
encoder = OneHotEncoder(sparse_output=False)  
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))  # Reshaping para que sea una matriz columna


# 4 División de Datos:


- División del dataset en conjuntos de entrenamiento y prueba utilizando train_test_split.

In [24]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

- Se imprimen las dimensiones de los datos de entrenamiento.

In [25]:
# Verificar las dimensiones de los datos cargados
print("Dimensiones de X_train:", X_train.shape) # xximágenes, cada una con xx píxeles
print("Dimensiones de y_train:", y_train.shape) # xx etiquetas en formato One-Hot 


Dimensiones de X_train: (56000, 784)
Dimensiones de y_train: (56000, 10)


# 5 Funciones de la Red Neuronal

- Implementación de funciones de activación (relu y softmax) y sus derivadas.
- Inicialización de los pesos y sesgos de la red neuronal.
- Implementación del pase hacia adelante (forward_pass) y la función de pérdida (cross-entropy).
- Implementación del algoritmo de retropropagación (backpropagation).

In [26]:
# Función de activación ReLU
def relu(x):
    return np.maximum(0, x)

# Derivada de ReLU
def relu_derivative(x):
    return (x > 0).astype(int)

# Función de activación Softmax para la capa de salida
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Derivada de Softmax
def softmax_derivative(x):
    return x * (1 - x)


In [27]:
# Parametros

# Inicialización de los pesos y sesgos
def initialize_parameters(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size, hidden_size) * 0.01  # Pesos de la capa de entrada a la capa oculta
    b1 = np.zeros((1, hidden_size))  # Sesgos de la capa oculta
    W2 = np.random.randn(hidden_size, output_size) * 0.01  # Pesos de la capa oculta a la capa de salida
    b2 = np.zeros((1, output_size))  # Sesgos de la capa de salida
    return W1, b1, W2, b2



In [28]:
# Forward y perdida
# Forward Pass
def forward_pass(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)  # Activación de la capa oculta
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)  # Activación de la capa de salida
    return A1, A2

# Función de pérdida (Cross-entropy)
def compute_loss(y, A2):
    m = y.shape[0]  # Número de muestras
    log_likelihood = -np.log(A2[range(m), np.argmax(y, axis=1)])
    loss = np.sum(log_likelihood) / m
    return loss



In [29]:
# Backpropagation

def backpropagation(X, y, A1, A2, W1, W2):
    m = X.shape[0]
    
    # Derivadas
    dZ2 = A2 - y  # Error en la capa de salida
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    
    dZ1 = np.dot(dZ2, W2.T) * relu_derivative(A1)  # Error en la capa oculta
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    
    return dW1, db1, dW2, db2



# 6 Entrenamiento de la Red:

- Entrenamiento de la red neuronal durante un número definido de épocas.
- Actualización de los parámetros (pesos y sesgos) utilizando el algoritmo de retropropagación.
- Impresión de la pérdida cada 100 épocas.

In [30]:
# Entrenamiento de la red
def train(X_train, y_train, input_size, hidden_size, output_size, epochs, learning_rate):
    W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size, output_size)
    
    for epoch in range(epochs):
        # Forward pass
        A1, A2 = forward_pass(X_train, W1, b1, W2, b2)
        
        # Cálculo de la pérdida
        loss = compute_loss(y_train, A2)
        
        # Backpropagation
        dW1, db1, dW2, db2 = backpropagation(X_train, y_train, A1, A2, W1, W2)
        
        # Actualización de parámetros
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")
    
    return W1, b1, W2, b2


In [31]:
# Parámetros de la red
input_size = 784  # Tamaño de la entrada (28x28 píxeles)
hidden_size = 128  # Número de neuronas en la capa oculta
output_size = 10  # Número de clases (diez categorías de ropa)
epochs = 300  # Número de épocas
learning_rate = 0.1  # Tasa de aprendizaje

# Entrenar el modelo
W1, b1, W2, b2 = train(X_train, y_train, input_size, hidden_size, output_size, epochs, learning_rate)

Epoch 0, Loss: 2.301905978634461
Epoch 100, Loss: 0.8962600999874162
Epoch 200, Loss: 0.7324050087507203


# 7 Evaluación del Modelo:

In [32]:
# Test para verificar que todo funciona
A1, A2 = forward_pass(X_test, W1, b1, W2, b2)
predictions = np.argmax(A2, axis=1)

# Calcular la precisión
accuracy = np.mean(predictions == np.argmax(y_test, axis=1)) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 77.41%


# Conclusión
He implementado una red neuronal desde cero para clasificar imágenes del dataset Fashion MNIST. He preprocesado los datos, definido las funciones necesarias para la red neuronal, entrenado el modelo y evaluado su precisión. Este proyecto ha permitido comprender los conceptos fundamentales de las redes neuronales, incluyendo la normalización de datos, la codificación One-Hot, el pase hacia adelante, la función de pérdida y la retropropagación. La precisión obtenida del modelo es del 77.41%, lo cual es un buen punto de partida para futuras mejoras y optimizaciones.